In [1]:
# 挂载
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 切换工作目录
import os

# 切换目录
%cd /content/drive/MyDrive/MyCode/data/KuaiRec/

# 确认一下是否成功切换，并看看里面有哪些文件
print("\n当前工作目录:")
!pwd
print("\n目录下的文件:")
!ls

/content/drive/MyDrive/MyCode/data/KuaiRec

当前工作目录:
/content/drive/MyDrive/MyCode/data/KuaiRec

目录下的文件:
data		 loaddata.py		 process_data.py
figs		 main_big.py		 process_data_v3.py
H3.py		 main_full.py		 processed_data
H3-v4.py	 main_worker.py		 run.ipynb
inspect_data.py  process_data_big.py	 Statistics_KuaiRec.ipynb
LICENSE		 process_data_hybrid.py  user-features.py


In [ ]:
# 运行检查数据的脚本
!python inspect_data.py

🔎 发现 7 个数据文件，准备逐一检查...

📄 文件名: big_matrix.csv
📌 列名 (8列):
['user_id', 'video_id', 'play_duration', 'video_duration', 'time', 'date', 'timestamp', 'watch_ratio']
👀 数据预览:
 user_id  video_id  play_duration  video_duration                    time     date    timestamp  watch_ratio
       0      3649          13838           10867 2020-07-05 00:08:23.438 20200705 1.593879e+09     1.273397
       0      9598          13665           10984 2020-07-05 00:13:41.297 20200705 1.593879e+09     1.244082
       0      5262            851            7908 2020-07-05 00:16:06.687 20200705 1.593879e+09     0.107613


📄 文件名: item_categories.csv
📌 列名 (2列):
['video_id', 'feat']
👀 数据预览:
 video_id    feat
        0     [8]
        1 [27, 9]
        2     [9]


📄 文件名: item_daily_features.csv
📌 列名 (58列):
['video_id', 'date', 'author_id', 'video_type', 'upload_dt', 'upload_type', 'visible_status', 'video_duration', 'video_width', 'video_height', 'music_id', 'video_tag_id', 'video_tag_name', 'show_cnt', 'show_use

In [ ]:
!python user-features.py

🕵️ 正在深度检查: ./data/user_features.csv

📌 包含 'follow' 关键词的列: ['follow_user_num', 'follow_user_num_range']

👀 关键列前 5 行预览:
   follow_user_num follow_user_num_range user_active_degree
0                5                (0,10]        high_active
1              386             (250,500]        full_active
2               27               (10,50]        full_active
3               16               (10,50]        full_active
4              122             (100,150]        full_active

DataType 检查:
 - follow_user_num: 值示例 '5' (类型: <class 'numpy.int64'>)
 - follow_user_num_range: 值示例 '(0,10]' (类型: <class 'str'>)

------------------------------------------------
结论预测：
如果值是 '5' (int)，那它只是统计数，不能用来建图。
如果值是 '[123, 456]' (str/list)，那它才是我们需要的一度人脉关系。


In [ ]:
# 数据预处理
!python process_data.py

开始全量处理数据...
--- 正在处理交互矩阵 (small_matrix.csv) ---
--- 正在构建 Author 映射 ---
--- 正在进行 ID Remapping ---
统计: Users=1411, Items=3327, Authors=2031
--- 正在处理 User Features ---
活跃度类别: ['UNKNOWN' 'full_active' 'high_active' 'middle_active']
缺失值默认将被编码为 ID: 0 (对应 label: UNKNOWN)
--- 正在处理 Social Graph ---
--- 正在计算 Author 分层 (Head/Mid/Tail) ---
Group 2 (Head): 406 人
Group 1 (Mid) : 609 人
Group 0 (Tail): 1016 人

数据处理完成！


In [ ]:
# worker
!python main_worker.py

🚀 运行环境: cuda
Loading data from ./processed_data/dataset.pkl...
构建稀疏邻接矩阵 (Normalized Adjacency Matrix)...
/content/drive/MyDrive/MyCode/data/KuaiRec/main_worker.py:90: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -0.5).flatten()
/content/drive/MyDrive/MyCode/data/KuaiRec/main_worker.py:103: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:654.)
  graph = torch.sparse.FloatTensor(i, v, torch.Size(shape)).to(config.device)
🔥 开始训练 (Epochs=50)...
Epoch  1 | Loss: 3.1955 | Recall@20: 0.0156 | NDCG@20: 0.3739 | Time: 199.9s
Epoch  2 | Loss: 3.0086 | Recall@20: 0.0152 | NDCG@20: 0.3706 | Time: 198.2s
Epoch  3 | Loss: 2.9549 | Recall@20: 0.0145 | NDCG@20: 0.3545 | Time: 198.6s
Epoch  4 | Loss: 2.9273 | Recall@20: 0.0141 | NDCG@20: 0.3447 | Time: 198.5s
Epoch  5 | Loss: 2.9101 | R

In [ ]:
# baseline-XSimGCL
!python main_worker.py

🚀 运行环境: cuda
Loading data from ./processed_data/dataset.pkl...
构建稀疏邻接矩阵 (Normalized Adjacency Matrix)...
/content/drive/MyDrive/MyCode/data/KuaiRec/main_worker.py:90: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -0.5).flatten()
/content/drive/MyDrive/MyCode/data/KuaiRec/main_worker.py:103: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:654.)
  graph = torch.sparse.FloatTensor(i, v, torch.Size(shape)).to(config.device)
🔥 开始训练 (Epochs=50)...
Epoch  1 | Loss: 3.2052 | Recall@20: 0.0157 | NDCG@20: 0.3759 | Time: 199.0s
Epoch  2 | Loss: 3.0350 | Recall@20: 0.0153 | NDCG@20: 0.3740 | Time: 197.4s
Epoch  3 | Loss: 2.9879 | Recall@20: 0.0144 | NDCG@20: 0.3519 | Time: 197.6s
Epoch  4 | Loss: 2.9638 | Recall@20: 0.0146 | NDCG@20: 0.3579 | Time: 197.7s
Epoch  5 | Loss: 2.9468 | R

In [ ]:
# 双层
!python main_full.py

🚀 Phase 2: Full Model Training on cuda
🔥 Start Joint Training (Worker + Manager)...
Epoch  1 | Loss: 3.2873 | Recall: 0.0125 | NDCG: 0.3129 | TailRatio: 1.0000
Epoch  2 | Loss: 3.0983 | Recall: 0.0122 | NDCG: 0.3077 | TailRatio: 1.0000
Epoch  3 | Loss: 3.0467 | Recall: 0.0114 | NDCG: 0.2863 | TailRatio: 1.0000
Epoch  4 | Loss: 3.0220 | Recall: 0.0109 | NDCG: 0.2745 | TailRatio: 1.0000
Epoch  5 | Loss: 3.0041 | Recall: 0.0107 | NDCG: 0.2682 | TailRatio: 1.0000
Epoch  6 | Loss: 2.9905 | Recall: 0.0109 | NDCG: 0.2751 | TailRatio: 1.0000
Epoch  7 | Loss: 2.9810 | Recall: 0.0112 | NDCG: 0.2807 | TailRatio: 1.0000
Epoch  8 | Loss: 2.9743 | Recall: 0.0110 | NDCG: 0.2792 | TailRatio: 1.0000
Epoch  9 | Loss: 2.9670 | Recall: 0.0110 | NDCG: 0.2779 | TailRatio: 1.0000
Epoch 10 | Loss: 2.9619 | Recall: 0.0111 | NDCG: 0.2789 | TailRatio: 1.0000
Epoch 11 | Loss: 2.9574 | Recall: 0.0108 | NDCG: 0.2719 | TailRatio: 1.0000
Epoch 12 | Loss: 2.9539 | Recall: 0.0109 | NDCG: 0.2741 | TailRatio: 1.0000
Epoc

In [ ]:
# 双层
!python H3.py

🚀 Phase 2 Full Model (H3-Score Ver.) running on cuda
🔥 Start Joint Training...
Epoch  1 | Loss: 3.6462 | NDCG: 0.3093 | Gini: 0.9180 | FanR: 0.0000 | H3-Score: 0.0000
Epoch  2 | Loss: 3.4584 | NDCG: 0.2938 | Gini: 0.9031 | FanR: 0.0000 | H3-Score: 0.0000
Epoch  3 | Loss: 3.4079 | NDCG: 0.2985 | Gini: 0.8966 | FanR: 0.0000 | H3-Score: 0.0000
Epoch  4 | Loss: 3.3815 | NDCG: 0.2954 | Gini: 0.8943 | FanR: 0.0000 | H3-Score: 0.0000
Epoch  5 | Loss: 3.3636 | NDCG: 0.2999 | Gini: 0.8907 | FanR: 0.0000 | H3-Score: 0.0000
Epoch  6 | Loss: 3.3501 | NDCG: 0.2949 | Gini: 0.8895 | FanR: 0.0000 | H3-Score: 0.0000
Epoch  7 | Loss: 3.3418 | NDCG: 0.2913 | Gini: 0.8893 | FanR: 0.0000 | H3-Score: 0.0000
Epoch  8 | Loss: 3.3331 | NDCG: 0.2924 | Gini: 0.8884 | FanR: 0.0000 | H3-Score: 0.0000
Epoch  9 | Loss: 3.3250 | NDCG: 0.2872 | Gini: 0.8884 | FanR: 0.0000 | H3-Score: 0.0000
Epoch 10 | Loss: 3.3202 | NDCG: 0.2769 | Gini: 0.8904 | FanR: 0.0000 | H3-Score: 0.0000
Epoch 11 | Loss: 3.3155 | NDCG: 0.2860 | 

In [ ]:
# 修改数据处理后运行 12-27-20:28
!python H3-v4.py

python3: can't open file '/content/H3-v4.py': [Errno 2] No such file or directory


In [ ]:
import pickle
import numpy as np
import os
import torch
from torch.utils.data import Dataset
import scipy.sparse as sp

# 1. 重新定义 Config，防止 NameError
class Config:
    dataset_path = './processed_data/dataset.pkl'
    device = 'cpu' # 诊断不需要 GPU

    # 占位参数，防止 Dataset 初始化报错
    epochs = 1
    batch_size = 2048
    lr_worker = 0.001
    lr_manager = 0.001
    weight_decay = 1e-4
    emb_dim = 64
    n_layers = 3
    cl_rate = 0.2
    eps = 0.1
    temp = 0.2
    eco_lambda = 0.5
    manager_scale = 0.1
    top_k = 20

# 2. 重新定义 RecDataset (精简版，只为读取社交数据)
class RecDataset(Dataset):
    def __init__(self, conf):
        if not os.path.exists(conf.dataset_path):
            raise FileNotFoundError(f"找不到文件: {conf.dataset_path}")

        with open(conf.dataset_path, 'rb') as f:
            data = pickle.load(f)

        # 尝试加载社交关系
        self.social_set = set()
        if 'social_edges' in data and len(data['social_edges']) > 0:
            for u, a in data['social_edges']:
                self.social_set.add((u, a))
        else:
            # 这种情况说明 dataset.pkl 里压根没存进去
            pass

    def __len__(self): return 0
    def __getitem__(self, idx): return 0

# 3. 执行诊断逻辑
print("正在检查数据集...")
try:
    conf = Config()
    dataset = RecDataset(conf)

    count = len(dataset.social_set)
    print(f"\n{'='*40}")
    print(f"📊 社交关系边数 (Social Edges): {count}")
    print(f"{'='*40}")

    if count == 0:
        print("✅ 诊断结论：FanR = 0 是正常的。")
        print("原因：你使用的是 Small Matrix (开发集)，数据太稀疏，")
        print("      导致用户关注的作者并没有包含在这个小数据集里。")
        print("建议：在汇报中直接说明“受限于开发集规模，FanR 暂时为 0，全量数据下将恢复正常”。")
    else:
        print("❌ 诊断结论：数据里有社交关系，但 FanR 依然为 0。")
        print("原因：可能是 evaluate 函数里的计算逻辑没写对，或者 ID 映射没对齐。")

except FileNotFoundError:
    print(f"❌ 错误：找不到 {Config.dataset_path}")
    print("请确认你之前运行过 process_data_v2.py 并且文件生成成功。")
except Exception as e:
    print(f"❌ 发生其他错误: {e}")

正在检查数据集...

📊 社交关系边数 (Social Edges): 50
❌ 诊断结论：数据里有社交关系，但 FanR 依然为 0。
原因：可能是 evaluate 函数里的计算逻辑没写对，或者 ID 映射没对齐。


In [ ]:
# 数据处理3
!python process_data.py

🚀 开始最终版数据处理 (Target: 捕获那50条有效边)...
--- [1/6] 读取交互矩阵 (small_matrix.csv) ---
--- [2/6] 构建 Item-Author 映射 ---
--- [3/6] ID 编码 ---
   统计: Users=1411, Items=3327, Authors=2031
--- [4/6] 用户活跃度特征 ---
--- [5/6] 提取社交关系 (User -> Author) ---
✅ 成功提取社交边: 50 条 (预期应接近 50)
--- [6/6] 创作者分层 ---
🎉 处理完成！dataset.pkl 已生成。


In [ ]:
import os
import pandas as pd

data_dir = './data'

print(f"📂 正在扫描目录: {data_dir} ...\n")

if os.path.exists(data_dir):
    files = sorted(os.listdir(data_dir))
    csv_files = [f for f in files if f.endswith('.csv')]

    if not csv_files:
        print("⚠️ 该目录下没有找到 CSV 文件。")

    for file_name in csv_files:
        file_path = os.path.join(data_dir, file_name)
        try:
            # 只读取第一行以获取表头，提高效率
            df = pd.read_csv(file_path, nrows=0)
            columns = df.columns.tolist()

            print(f"📄 文件名: {file_name}")
            print(f"📌 字段 ({len(columns)}列): {columns}")
            print("-" * 50)
        except Exception as e:
            print(f"❌ 无法读取 {file_name}: {e}")
else:
    print(f"❌ 目录不存在: {data_dir}")

📂 正在扫描目录: ./data ...

📄 文件名: big_matrix.csv
📌 字段 (8列): ['user_id', 'video_id', 'play_duration', 'video_duration', 'time', 'date', 'timestamp', 'watch_ratio']
--------------------------------------------------
📄 文件名: item_categories.csv
📌 字段 (2列): ['video_id', 'feat']
--------------------------------------------------
📄 文件名: item_daily_features.csv
📌 字段 (58列): ['video_id', 'date', 'author_id', 'video_type', 'upload_dt', 'upload_type', 'visible_status', 'video_duration', 'video_width', 'video_height', 'music_id', 'video_tag_id', 'video_tag_name', 'show_cnt', 'show_user_num', 'play_cnt', 'play_user_num', 'play_duration', 'complete_play_cnt', 'complete_play_user_num', 'valid_play_cnt', 'valid_play_user_num', 'long_time_play_cnt', 'long_time_play_user_num', 'short_time_play_cnt', 'short_time_play_user_num', 'play_progress', 'comment_stay_duration', 'like_cnt', 'like_user_num', 'click_like_cnt', 'double_click_cnt', 'cancel_like_cnt', 'cancel_like_user_num', 'comment_cnt', 'comment_user_num',

In [ ]:
import os

file_path = 'process_data_v3.py'

# 定义修复逻辑
def apply_fix(content):
    new_lines = []
    lines = content.split('\n')

    merged_added = False

    for line in lines:
        new_lines.append(line)

        # 找到读取 small_matrix.csv 的地方
        if "pd.read_csv" in line and "small_matrix.csv" in line and not merged_added:
            # 在这一行下面插入合并代码
            indent = line[:len(line) - len(line.lstrip())] # 获取缩进

            patch_code = [
                f"{indent}# --- [Auto-Fix] Merge author_id from item_features ---",
                f"{indent}print('正在合并 Author ID 信息...')",
                f"{indent}df_feat = pd.read_csv(os.path.join(RAW_DATA_PATH, 'item_daily_features.csv'))",
                f"{indent}# 只保留需要的列以节省内存",
                f"{indent}df_author_map = df_feat[['video_id', 'author_id']].drop_duplicates()",
                f"{indent}# 合并到主表",
                f"{indent}df_matrix = df_matrix.merge(df_author_map, on='video_id', how='left')",
                f"{indent}# 填充找不到作者的视频 (可选)",
                f"{indent}df_matrix['author_id'] = df_matrix['author_id'].fillna(-1).astype(int)",
                f"{indent}print(f'合并后列名: {{df_matrix.columns.tolist()}}')",
                f"{indent}# --------------------------------------------------"
            ]
            new_lines.extend(patch_code)
            merged_added = True

    return '\n'.join(new_lines)

# 执行修复
if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    if "item_daily_features.csv" in content:
        print("⚠️ 代码似乎已经包含读取 item_daily_features 的逻辑，跳过修复，请检查是否位置不对。")
    else:
        new_content = apply_fix(content)
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(new_content)
        print("✅ 已成功修改 process_data_v3.py，增加了 Author ID 合并逻辑。")
else:
    print(f"❌ 找不到文件: {file_path}")

In [ ]:
# 再次尝试运行数据处理
!python process_data_v3.py

In [ ]:
import pandas as pd
import os
from ast import literal_eval

# ================= 配置 =================
DATA_DIR = './data'  # 请确认路径
MATRIX_FILE = 'small_matrix.csv' # 先测 small，测完可以换 big_matrix.csv 试试
# MATRIX_FILE = 'big_matrix.csv'

def check_raw_intersection():
    print(f"🕵️‍♂️ 正在检查原始数据交集 (基于 {MATRIX_FILE})...")

    # 1. 加载核心交互数据 (Small Matrix)
    # 我们只关心在这个矩阵里出现过的 User 和 Item
    df_matrix = pd.read_csv(os.path.join(DATA_DIR, MATRIX_FILE))
    # 过滤无效交互 (保持和你处理代码一致)
    df_matrix = df_matrix[df_matrix['watch_ratio'] >= 0.5]

    valid_users = set(df_matrix['user_id'].unique())
    valid_videos = set(df_matrix['video_id'].unique())

    print(f"1. 交互矩阵 ({MATRIX_FILE}):")
    print(f"   - 有效用户数: {len(valid_users)}")
    print(f"   - 有效视频数: {len(valid_videos)}")

    # 2. 加载作者信息 (Item -> Author)
    # 只有发了这些 valid_videos 的作者，才是我们模型里的“有效作者”
    feat_path = os.path.join(DATA_DIR, 'item_daily_features.csv')
    if not os.path.exists(feat_path):
        print(f"❌ 找不到 {feat_path}，无法关联作者！")
        return

    df_feat = pd.read_csv(feat_path, usecols=['video_id', 'author_id'])
    # 只保留 valid_videos 的记录
    df_feat = df_feat[df_feat['video_id'].isin(valid_videos)]

    valid_authors = set(df_feat['author_id'].unique())
    print(f"2. 作者信息:")
    print(f"   - 关联到的有效作者数: {len(valid_authors)}")

    if len(valid_authors) == 0:
        print("❌ 警告：交互矩阵里的视频都没有找到对应的作者！检查 video_id 是否匹配。")
        return

    # 3. 加载社交网络
    # 我们要看：valid_users 关注的人里，有多少是 valid_authors？
    social_path = os.path.join(DATA_DIR, 'social_network.csv')
    df_social = pd.read_csv(social_path)

    # 只看 valid_users 的社交关系
    df_social = df_social[df_social['user_id'].isin(valid_users)]

    print(f"3. 社交网络:")
    print(f"   - 包含有效用户的社交记录数: {len(df_social)}")

    # 4. 核心碰撞 (Intersection)
    print("--- 正在计算交集 (这可能需要几秒钟) ---")

    total_edges = 0
    valid_edges = 0

    # 为了加速，把 valid_authors 变成字符串集合 (以防 ID 类型不一致)
    valid_authors_str = set(map(str, valid_authors))

    for _, row in df_social.iterrows():
        try:
            friend_list = literal_eval(row['friend_list'])
            # 统计
            for friend_id in friend_list:
                total_edges += 1
                # 检查 friend_id (被关注者) 是否在 valid_authors (有效作者) 里
                if str(friend_id) in valid_authors_str:
                    valid_edges += 1
        except:
            continue

    print(f"\n📊 诊断结果:")
    print(f"   - 用户总关注数 (Raw Edges): {total_edges}")
    print(f"   - **有效重合关注数 (Valid Overlap)**: {valid_edges}")

    if valid_edges == 0:
        print("\n❌ 结论: 悲剧了。")
        print("   在 Small Matrix 的用户和作者之间，**没有任何** 关注关系。")
        print("   原因: Small Matrix 采样太稀疏，刚好把社交圈切断了。")
        print("   对策: 必须切换到 Big Matrix，或者手动注入假数据。")
    else:
        print(f"\n✅ 结论: 有希望！")
        print(f"   理论上应该有 {valid_edges} 条社交边被提取出来。")
        print("   如果 process_data 跑出来是 0，那就是代码 Bug (ID 映射问题)。")
        print("   如果 process_data 跑出来接近这个数，那就是正常的。")

if __name__ == '__main__':
    check_raw_intersection()

🕵️‍♂️ 正在检查原始数据交集 (基于 small_matrix.csv)...
1. 交互矩阵 (small_matrix.csv):
   - 有效用户数: 1411
   - 有效视频数: 3327
2. 作者信息:
   - 关联到的有效作者数: 2031
3. 社交网络:
   - 包含有效用户的社交记录数: 146
--- 正在计算交集 (这可能需要几秒钟) ---

📊 诊断结果:
   - 用户总关注数 (Raw Edges): 214
   - **有效重合关注数 (Valid Overlap)**: 50

✅ 结论: 有希望！
   理论上应该有 50 条社交边被提取出来。
   如果 process_data 跑出来是 0，那就是代码 Bug (ID 映射问题)。
   如果 process_data 跑出来接近这个数，那就是正常的。


In [8]:
# 全量数据集
!python process_data_big.py

🚀 开始全量数据处理 (Big Matrix)...
⚠️ 注意：全量数据量大，请关注 Colab 内存使用情况
--- [1/6] 读取交互矩阵 (big_matrix.csv) ---
    - 原始: 12530806 -> 过滤后: 8031791
--- [2/6] 构建 Item-Author 映射 ---
--- [3/6] ID 编码 ---
   统计: Users=7176, Items=10727, Authors=8369
--- [4/6] 用户活跃度特征 ---
--- [5/6] 提取社交关系 (Real Data Only) ---
    正在扫描 472 个用户的关注列表...
✅ 成功提取真实社交边: 670 条
--- [6/6] 分层与保存 ---
🎉 全量数据处理完成！保存为: dataset_big.pkl


In [4]:
# 隐式关注关系
!python process_data_hybrid.py

🚀 开始全量数据处理 (Hybrid: 显式 + 隐式挖掘)...
   策略: Top-15 且 互动次数>=2
--- [1/6] 读取交互矩阵 (big_matrix.csv) ---
    原始数据: 12530806 -> 过滤后有效交互: 8031791
--- [2/6] 关联作者信息 ---
    关联作者后剩余交互: 8031791
--- [3/6] ID 编码 ---
    Users: 7176, Items: 10727, Authors: 8369
--- [4/6] 构建混合社交网络 ---
    A. 挖掘隐式交互 (Top-15, MinCount>=2)...
       正在聚合 User-Author 交互频次...
       过滤低频交互后，剩余候选对: 1478323
       >>> 挖掘出隐式边数: 106954
    B. 提取显式关注 (social_network.csv)...
       正在扫描 472 个用户的关注列表...
       >>> 提取出显式边数: 670
    C. 合并去重...
✅ 混合社交网络构建完成！
   最终总边数: 107620
   平均每人关注: 15.00 个作者
--- [5/6] 特征处理 ---
--- [6/6] 切分数据集与保存 ---
🎉 处理完成！文件已保存至: ./processed_data/dataset_hybrid.pkl
